# ICAMP Model (Merton, 1973) #

### Adding a Hedge Portfolio to the CAPM model ###

In [315]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Statistics
import statsmodels
import statsmodels.api as sm
from arch import arch_model

# Pretty Notation
from IPython.display import display, Math

In [316]:
# Create the Weights function
def wexp(N, half_life):
    c = np.log(0.5)/half_life
    n = np.array(range(N))
    w = np.exp(c*n)
    return np.flip(w/np.sum(w))

In [317]:
# Import Data
ticker = 'AAPL'

# Stock Data (Adjust Ticker)
df_stock = pd.read_csv(rf"..\stocks\{ticker}.csv")
df_stock = df_stock.set_index('Date')
df_stock.index = pd.to_datetime(df_stock.index)

# Get the important data for the Risk Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)
rfr.dropna(inplace = True)

# Get the important data for the S&P500
sp500 = pd.read_csv(r"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

# Get the important data for the Zero Beta Portfolio
zero_beta = pd.read_csv(r"..\additional_data\zero_beta.csv")
zero_beta = zero_beta.set_index('Date')
zero_beta.index = pd.to_datetime(zero_beta.index)

In [318]:
# Create the DataFrame
data = pd.DataFrame()

# Create the Columns
data['stock_returns'] = df_stock['Adjusted_close'].pct_change(1)
data['risk_free_rate'] = (((1 + (rfr['risk_free_rate'].div(100)))**(1/360)) - 1)
data['benchmark_returns'] = sp500['sp_500'].pct_change(1)
data['zero_beta_returns'] = zero_beta

# Create the Excess Returns
data['stock_excess_returns'] = data['stock_returns'] - data['risk_free_rate']
data['benchmark_excess_returns'] = data['benchmark_returns'] - data['risk_free_rate']
data['hedge_port_excess_returns'] = data['zero_beta_returns'] - data['risk_free_rate']

# Multiply by 100
data = data.mul(100)

data.dropna(inplace = True)
data

In [319]:
# Check Correlations Matrix

data[['stock_excess_returns', 'benchmark_excess_returns', 'hedge_port_excess_returns']].corr()

In [320]:
# Calculate the ICAPM with the whole time stamp

y = data['stock_excess_returns']

x = data[['benchmark_excess_returns', 'hedge_port_excess_returns']]
x = sm.add_constant(x)

# Calculate Weights
window = len(y)
weights = window * wexp(window, window/2)

#Model specification
model = sm.WLS(
    y, 
    x,
    missing='drop',
    weights=weights,
    )   
     
#the results of the model
results = model.fit() 
    
#here we check the summary
print(results.summary())   

In [321]:
# Set rolling window size
window = 252
weights = window * wexp(window, window/2)

# Lists to store rolling coefficients
betas = []
index = []
lower_bounds = []
upper_bounds = []

# Rolling regression
for i in range(window, len(data)):
    Y_window = y.iloc[i - window:i]
    X_window = x.iloc[i - window:i]
    
    X_window = sm.add_constant(X_window)

    # Fit WLS model
    model = sm.WLS(
        Y_window, 
        X_window, 
        missing='drop', 
        weights=weights,
    ).fit()

    # Store coefficients (const, X1, X2)
    betas.append(model.params.values)
    index.append(data.index[i])  # Use the last date of the window

    # Store lower and upper bounds of 95% confidence intervals
    ci = model.conf_int(alpha=0.05)  # 95% CI
    lower_bounds.append(ci.iloc[:, 0].values)  # First column: lower bound
    upper_bounds.append(ci.iloc[:, 1].values)  # Second column: upper bound


In [322]:
# Convert list of coefficients to DataFrame
betas_df = pd.DataFrame(betas, columns=x.columns, index=index)

betas_df

In [323]:
# Lower bounds DataFrame
lower_df = pd.DataFrame(lower_bounds, columns=[f'{col}_lower' for col in x.columns], index=index)

lower_df

In [324]:
# Upper bounds DataFrame
upper_df = pd.DataFrame(upper_bounds, columns=[f'{col}_upper' for col in x.columns], index=index)

upper_df

In [325]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_df['benchmark_excess_returns'], label='Market Beta', color='orange', alpha=0.7)
plt.plot(betas_df['hedge_port_excess_returns'], label='Hedge Portfolio Beta', color='blue', alpha=0.7)
plt.plot(betas_df['const'], label='Alpha', color='green', alpha=0.7)

# Config
plt.title('Beta vs Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

In [326]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Unemployment Rate Plot
betas_df['benchmark_excess_returns'].plot(color = 'blue', ax = ax1, alpha=0.7)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Market Beta', 
    color='blue'
    )

# Unemployment Mentions Plot
ax2 = ax1.twinx()

betas_df['hedge_port_excess_returns'].plot(color = 'orange', ax = ax2, alpha=0.7)
ax2.set_ylabel(
    'Hedge Portfolio Beta', 
    color='orange'
    )

plt.title('Beta vs Beta Time Series')
plt.show()

In [327]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Unemployment Rate Plot
betas_df['const'].plot(color = 'limegreen', ax = ax1, alpha=0.7)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Alpha', 
    color='green'
    )

# Unemployment Mentions Plot
ax2 = ax1.twinx()

betas_df['hedge_port_excess_returns'].plot(color = 'orange', ax = ax2, alpha=0.7)
ax2.set_ylabel(
    'Hedge Portfolio Beta', 
    color='orange'
    )

plt.title('Beta vs Beta Time Series')
plt.show()

In [328]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_df['benchmark_excess_returns'], label='Market Beta', color='black', alpha=0.7)
plt.fill_between(upper_df.index, lower_df['benchmark_excess_returns_lower'], upper_df['benchmark_excess_returns_upper'], color='skyblue', alpha=0.2, label='95% CI')

# Config
plt.title('Market Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

In [329]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(betas_df['hedge_port_excess_returns'], label='Hedge Portfolio Beta', color='black', alpha=0.7)
plt.fill_between(upper_df.index, lower_df['hedge_port_excess_returns_lower'], upper_df['hedge_port_excess_returns_upper'], color='peachpuff', alpha=0.5, label='95% CI')
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Hedge Portfolio Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Betas')
plt.legend()

# Show
plt.show()

### Adding Constrains to the Model ###

In [331]:
# Remembering the Form of the Constrained OLS

display(Math(r"\beta=(X^⊤X)^{-1}(X^⊤Y)-P"))
display(Math(r"P=\frac{R^⊤(X^⊤X)^{-1}(X^⊤Y)-q}{R^⊤(X^⊤X)^{-1}R}(X^⊤X)^{-1}R"))

In [332]:
#Show the betas

model_betas = results.params

model_betas

In [333]:
# Make the Models using GLM
OLS_from_GLM = sm.GLM(y, x)

result_c = OLS_from_GLM.fit_constrained('benchmark_excess_returns + hedge_port_excess_returns = 1')

#here we check the summary
print(result_c.summary())   